In [341]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import uuid

In [347]:
# HELPER FUNCTIONS


def calculate_relative_value(df, verbose=False):
    position_mean = df['FANTASYPTS'].mean()
    position_stddev = np.std(df['FANTASYPTS'])

    for index, row in df.iterrows():
        df.loc[index, 'relative_value'] = ((df.loc[index, 'FANTASYPTS'] - position_mean)/position_stddev) + 1 # +1 transformation on the end of this zscore calculation is to reduce the number of negative values - this is important because later the relative values will be adjusted, and I want the distribution of zscores to be preserved, but also to lie mostly above 0
        if verbose:
            print(f"Index: {index}, RK: {row['RK']}, Pts: {row['FANTASYPTS']}, Position: {row['position']}, Mean: {position_mean}, Stddev: {position_stddev}, Zscore: {df.loc[index, 'relative_value']}")
    return(df)

def relative_value_adjustment(df, adjustment, verbose=False):
    # adjust the relative values so that QBS, WRS, and RBS are emphasized in the beginning rounds and kickers and defenses are taken as late as possible
    # adjustment should be list of len(nunique(position)) - we have 6 positions, so should be length 6

    if verbose:
        print(f'QB adjustment: {adjustment[0]}\nWR adjustment: {adjustment[1]}\nRB adjustment: {adjustment[2]}\nTE adjustment: {adjustment[3]}\nK adjustment: {adjustment[4]}\nDST adjustment: {adjustment[5]}\n')
    
    
    # make relative value adjustments by position
    df.loc[df['position'] == 'QB', 'relative_value'] = (df.loc[df['position'] == 'QB', 'relative_value'] * adjustment[0])
    df.loc[df['position'] == 'WR', 'relative_value'] = (df.loc[df['position'] == 'WR', 'relative_value'] * adjustment[1])
    df.loc[df['position'] == 'RB', 'relative_value'] = (df.loc[df['position'] == 'RB', 'relative_value'] * adjustment[2])
    df.loc[df['position'] == 'TE', 'relative_value'] = (df.loc[df['position'] == 'TE', 'relative_value'] * adjustment[3])
    df.loc[df['position'] == 'K', 'relative_value'] = (df.loc[df['position'] == 'K', 'relative_value'] * adjustment[4])
    df.loc[df['position'] == 'DST', 'relative_value'] = (df.loc[df['position'] == 'DST', 'relative_value'] * adjustment[5])

    return(df)


def impute_and_smooth(df, verbose=False):
    # need to impute some incorrect values

    # FANTASYPTS is the number of fantasy points a player scored last season (2024), so rookies have 0. Also players who were injured
    # suffer, for example CMC has only 40.3 points. 


    # here I am imputing 0 values (rookies) based on the average of the 4 closest ranked players

    for index, row in df.iterrows():

        # get closest_4 values 
        if index == 0:
            closest_4 = [1, 2, 3, 4]
        elif index == 1:
            closest_4 = [0, 2, 3, 4]
        elif index == (len(df['RK']) - 1):
            closest_4 = [index-4, index-3, index-2, index-1]
        elif index == (len(df['RK']) - 2):
            closest_4 = [index-4, index-3, index-2, index]
        else:
            closest_4 = [index+1, index+2, index-1, index-2]

        if row['FANTASYPTS'] == 0:
            knearest = (df.loc[closest_4[0], 'FANTASYPTS'] + df.loc[closest_4[1], 'FANTASYPTS'] + 
                        df.loc[closest_4[2], 'FANTASYPTS'] + df.loc[closest_4[3], 'FANTASYPTS'])/4
            df["FANTASYPTS"] = df["FANTASYPTS"].astype(float) # get rid of warning - not sure when FANTASYPTS is not a float but this fixes
            df.loc[index, "FANTASYPTS"] = round(knearest, 1)

    # I want to smooth the curve and minimize outliers to fix things like CMC pts score. Therefore I will calculate the average of the 4
    # closest players and if the player in question is > 1.5 standard devations away from the mean of those players, I will set their
    # FANTASYPTS score to the mean of those 4 players. This has to be done before merging the positions because it will be easier to handle
    # the beginning and end of the lists. One caveat to this is that I do not want to mimimize high outliers, just low outliers.

    for index, row in df.iterrows():

        # get closest_4 values 
        if index == 0:
            closest_4 = [1, 2, 3, 4]
        elif index == 1:
            closest_4 = [0, 2, 3, 4]
        elif index == (len(df['RK']) - 1):
            closest_4 = [index-4, index-3, index-2, index-1]
        elif index == (len(df['RK']) - 2):
            closest_4 = [index-4, index-3, index-2, index]
        else:
            closest_4 = [index+1, index+2, index-1, index-2]
        
        # calculate mean, stddev, zscore
        mean = round((df.loc[closest_4[0], 'FANTASYPTS'] + df.loc[closest_4[1], 'FANTASYPTS'] + 
                df.loc[closest_4[2], 'FANTASYPTS'] + df.loc[closest_4[3], 'FANTASYPTS'])/4, 1)
        std_dev = round((((df.loc[closest_4[0], 'FANTASYPTS'] - mean)**2 + (df.loc[closest_4[1], 'FANTASYPTS'] - mean)**2 +
                (df.loc[closest_4[2], 'FANTASYPTS'] - mean)**2 + (df.loc[closest_4[3], 'FANTASYPTS'] - mean)**2)/3)**0.5, 2)
        zscore = round((df.loc[index, 'FANTASYPTS'] - mean)/std_dev, 2)

        if verbose:
            print(f"Index: {index}, RK: {row['RK']}, Pts: {row['FANTASYPTS']}, Position: {row['position']}, Mean of 4 closest: {mean}, Stddev: {std_dev}, Zscore: {zscore}")

        if zscore < -2.4:
            if verbose:
                print(f'Need to replace rank {row["RK"]} points ({row["FANTASYPTS"]}) with mean: {mean}')
            df["FANTASYPTS"] = df["FANTASYPTS"].astype(float) # get rid of warning - not sure when FANTASYPTS is not a float but this fixes
            df.loc[index, "FANTASYPTS"] = mean

    return df

def pick_player(players, team, num_players_allowed):
    max_num_players_allowed = len(players)-1#define max for recursive search
    if num_players_allowed > max_num_players_allowed:
        num_players_allowed = max_num_players_allowed

    pick_selection = True

    while pick_selection:   # make sure that a pick is valid before modifying teams
        if num_players_allowed > max_num_players_allowed:
            num_players_allowed = max_num_players_allowed
        pick = random.randint(0,num_players_allowed-1)

        options = players.iloc[0:num_players_allowed]

        if team.starting_positions_needed.size != 0:
            if options.iloc[pick]['position'] in team.starting_positions_needed:
                pick_selection = False
            else:
                num_players_allowed += 1
        elif team.bench_positions_needed.size != 0:
            if options.iloc[pick]['position'] in team.bench_positions_needed:
                pick_selection = False
            else:
                num_players_allowed += 1
        elif team.bench_positions_needed.size == 0 and team.starting_positions_needed.size == 0:
            pick_selection = False
    
    if team.starting_positions_needed.size != 0: # if there are starting positions left
        if options.iloc[pick]['position'] in team.starting_positions_needed: # if the pick has a position in starting_positions
            team.add_drafted_player(options.iloc[pick]) # add player
            team.add_points(options.iloc[pick]['FANTASYPTS']) # add score
            team.remove_starting_position_needed(options.iloc[pick]['position']) # remove option from starting_positions
            players.drop(pick, axis='index', inplace=True) # remove picked player from the draft pool
            return players, team
        else:
            pass
    elif team.bench_positions_needed.size != 0: # if there are bench positions left
        if options.iloc[pick]['position'] in team.bench_positions_needed: # if the pick has a position in bench_positions
            team.add_drafted_player(options.iloc[pick]) # add player
            team.add_points(options.iloc[pick]['FANTASYPTS'] * 0.2) # add score * 0.2 - this is a bench position, so their total value should be smaller compared to starting players
                                                                    # this can be modulated to give bench players more of an impact
                                                                    # Also affects kickers and defense, but because I don't want them to account for
                                                                    # much in the simulation, that is fine.

            team.remove_bench_position_needed(options.iloc[pick]['position']) # remove option from bench_positions
            players.drop(pick, axis='index', inplace=True) # remove picked player from the draft pool
            return players, team
        else:
            pass
    elif team.bench_positions_needed.size == 0 and team.starting_positions_needed.size == 0: # if there are no bench or starting positions then return
        return players, team
    
    return players, team # return if no conditions are met

def run_monte_carlo(player_dataset, n_trials=10000000):  # 10 million default trials

    new_dataset = pd.DataFrame()

    for trial in range(n_trials):
        if (trial % 100000) == 0:
            print(f"{(trial/n_trials)*100}% complete...")
        trial_dataset = player_dataset.copy()
        
        draft = Draft()
        draft.run_draft(trial_dataset)

        for team in draft.league.teams:
            new_row = pd.DataFrame([{'team_id': uuid.uuid4(), 'sim_id': trial, 'draft_position': team.draft_pos, 'proj_total_points': team.season_points_total, 
                                     'proj_position': team.league_position, 'positions': team.position_picked_list, 
                                     'players': team.player_picked_list}])
            new_dataset = pd.concat([new_dataset, new_row], ignore_index=True)
    
    return new_dataset



In [348]:
#CLASS DEFINITIONS

class Team:
    def __init__(self):
        self.starting_positions_needed = np.array(['QB', 'WR', 'WR', 'RB', 'RB', 'TE']) #list of positions needed to draft full team - will remove items from list as team grows
        self.bench_positions_needed = np.array(['QB', 'WR', 'WR', 'RB', 'RB', 'K', 'DST']) #list of positions needed to draft full team - will remove items from list as team grows

        self.draft_pos = 0 # integer - 1 <= draft_pos <= number of teams in draft

        self.season_points_total = 0 # sum of total projected points for the season - running tally as draft goes on
        self.league_position = 0 # final league position - integer between 1 and number of teams (inclusive)

        self.position_picked_list = np.array([])     # 1 = QB, 2 = WR, 3 = RB, 4 = TE, 5 = K, 6 = DEF 
        self.player_picked_list = np.array([])
    
    #add points to season_points_total - will use to calculate league position - add points after every player drafted
    def add_points(self, points):
        self.season_points_total += points
    
    # remove position from starting_positions_needed - will need to keep track of what to draft
    def remove_starting_position_needed(self, pos):
        
        matches = self.starting_positions_needed == pos

        #gets index if if finds match, gets -1 otherwise
        if np.any(matches):
            index = int(np.where(matches)[0][0])
        else:
            index = -1

        self.starting_positions_needed = np.delete(self.starting_positions_needed, index)

    # remove position from bench_positions_needed - will need to keep track of what to draft
    def remove_bench_position_needed(self, pos):
        
        matches = self.bench_positions_needed == pos

        #gets index if if finds match, gets -1 otherwise
        if np.any(matches):
            index = int(np.where(matches)[0][0])
        else:
            index = -1

        self.bench_positions_needed = np.delete(self.bench_positions_needed, index)

    # set the league position based on season_points_total
    def set_league_position(self, pos):
        self.league_position = pos

    # set draft position
    def set_draft_position(self, pos):
        self.draft_pos = pos
    
    # add player position to draft list
    def add_drafted_player(self, player):
        self.position_picked_list = np.append(self.position_picked_list, player['position'])
        self.player_picked_list = np.append(self.player_picked_list, player['id'])


class League:
    def __init__(self, num_teams='10team'):
        self.num_teams = num_teams

        if num_teams == '12team': # generate 12 team league
            team1 = Team()
            team2 = Team()
            team3 = Team()
            team4 = Team()
            team5 = Team()
            team6 = Team()
            team7 = Team()
            team8 = Team()
            team9 = Team()
            team10 = Team()
            team11 = Team()
            team12 = Team()
            self.teams = [team1, team2, team3, team4, team5, team6, team7, team8, team9, team10, team11, team12]
        else: #generate 10 team league
            team1 = Team()
            team2 = Team()
            team3 = Team()
            team4 = Team()
            team5 = Team()
            team6 = Team()
            team7 = Team()
            team8 = Team()
            team9 = Team()
            team10 = Team()            
            self.teams = [team1, team2, team3, team4, team5, team6, team7, team8, team9, team10]

    def initialize_draft_positions(self):
        num_teams = len(self.teams) # get number of teams in draft
        draft_order = list(range(1, num_teams+1)) # set list of draft order
        random.shuffle(draft_order) # shuffle list of draft order
        for i in range(num_teams):
            self.teams[i].set_draft_position(draft_order[i]) #sequentially set draft order
        self.teams.sort(key=lambda x: x.draft_pos, reverse=False) # sort teams by draft positions
    
    def set_league_positions(self):
        self.teams.sort(key=lambda x: x.season_points_total, reverse=False) # sort teams by points
        for i in range(len(self.teams)):
            self.teams[i].league_position = i + 1


class Draft: # class for drafting teams
    def __init__(self, num_teams='10team'):
        self.rounds = 13
        self.league = League(num_teams)
        self.league.initialize_draft_positions()

    def run_draft(self, players): # run draft based on passed in player dataset
        for i in range(self.rounds):
            for team in self.league.teams:  
                result = pick_player(players, team, 5)
                players = result[0].reset_index(drop=True)
        self.league.set_league_positions()
    
    def print_teams(self): # print the teams created from the draft
        for team in self.league.teams:
            print(f"Players picked: {team.player_picked_list}")
            print(f"Positions picked: {team.position_picked_list}")

In [349]:
# Dataset creation
# Source: https://www.fantasypros.com/nfl/rankings/half-point-ppr-cheatsheets.php


# have to merge all positions together because the original dataset didn't have position :/
qbs = pd.read_csv('FantasyPros_2025_Draft_QB_Rankings.csv')
qbs['position'] = 'QB'
qbs = qbs[['RK', 'FANTASYPTS', 'position']]


wrs = pd.read_csv('FantasyPros_2025_Draft_WR_Rankings.csv')
wrs['position'] = 'WR'
wrs = wrs[['RK', 'FANTASYPTS', 'position']]


rbs = pd.read_csv('FantasyPros_2025_Draft_RB_Rankings.csv')
rbs['position'] = 'RB'
rbs = rbs[['RK', 'FANTASYPTS', 'position']]


tes = pd.read_csv('FantasyPros_2025_Draft_TE_Rankings.csv')
tes['position'] = 'TE'
tes = tes[['RK', 'FANTASYPTS', 'position']]


ks = pd.read_csv('FantasyPros_2025_Draft_K_Rankings.csv')
ks['position'] = 'K'
ks = ks[['RK', 'FANTASYPTS', 'position']]


dst = pd.read_csv('FantasyPros_2025_Draft_DST_Rankings.csv')
dst['position'] = 'DST'
dst = dst[['RK', 'FANTASYPTS', 'position']]

# impute and smooth the distribution
qbs_smoothed = impute_and_smooth(qbs, verbose=False)
wrs_smoothed = impute_and_smooth(wrs, verbose=False)
rbs_smoothed = impute_and_smooth(rbs, verbose=False)
tes_smoothed = impute_and_smooth(tes, verbose=False)
ks_smoothed = impute_and_smooth(ks, verbose=False)
dst_smoothed = impute_and_smooth(dst, verbose=False)

#calculate relative values based on smoothed distribution
qbs_rv = calculate_relative_value(qbs_smoothed, verbose=False)
wrs_rv = calculate_relative_value(wrs_smoothed, verbose=False)
rbs_rv = calculate_relative_value(rbs_smoothed, verbose=False)
tes_rv = calculate_relative_value(tes_smoothed, verbose=False)
ks_rv = calculate_relative_value(ks_smoothed, verbose=False)
dst_rv = calculate_relative_value(dst_smoothed, verbose=False)

players = pd.concat([qbs_rv, wrs_rv, rbs_rv, tes_rv, ks_rv, dst_rv]).reset_index(drop=True)
players['RK'] = players['RK'].astype(str) # change type of RK to string


# define importance of each position - (QB, WR, RB, TE, K, DST)
position_importance = [0.9, 1, 1, 0.72, 0.2, 0.2] # QB = 0.9, WR = 1, RB = 1, TE = 0.72, K = 0.2, DST = 0.2


players_adjusted = relative_value_adjustment(players, position_importance, verbose=True)
#players_adjusted['id'] = str(players_adjusted['position']) + str(players_adjusted['RK'])
players_adjusted['id'] = players_adjusted['position'].str.cat(players_adjusted['RK'], sep='')

players_adjusted = players_adjusted.sort_values(by='relative_value', ascending=False).reset_index(drop=True)

players_adjusted = players_adjusted.drop(['RK'], axis=1) # drop rank


players_adjusted.head(5)

#for index, row in df_sorted.iterrows():
    #print(f"Index: {index}, RK: {row['RK']}, Pts: {row['FANTASYPTS']}, Position: {row['position']}, relative value: {row['relative_value']}")


QB adjustment: 0.9
WR adjustment: 1
RB adjustment: 1
TE adjustment: 0.72
K adjustment: 0.2
DST adjustment: 0.2



,FANTASYPTS,position,relative_value,id
0,339.5,WR,4.897355,WR1
1,338.8,RB,4.005781,RB3
2,336.9,RB,3.983176,RB2
3,326.9,RB,3.864204,RB6
4,266.0,WR,3.787193,WR2


In [345]:
players = players_adjusted.copy()

draft_data = run_monte_carlo(players) # run simulation! 10 million times :)
print("\nSimulation complete.\n")

print(draft_data.info())
draft_data.to_csv('draft_data.csv') # save draft data as a csv



0.0% complete...


KeyboardInterrupt: 